<a href="https://colab.research.google.com/github/Anuragpandey2005/LoanTap_claasification/blob/main/LoadTap_logistiic_regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

# Import data with required libraries

In [ ]:
import gdown
file_id = "1ZPYj7CZCfxntE8p2Lze_4QO4MyEOy6_d"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.csv", quiet=False)
df = pd.read_csv("data.csv")

In [ ]:
df.head()

# About the data

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

# We got null values in 'mort_acc','emp_title','emp_length','pub_rec_bankruptcies','title'

In [ ]:
df.nunique()

In [ ]:
df.groupby("emp_length")["loan_status"].value_counts(normalize=True)* 100

# Remove not  important columns from dataset

columns why not important

* emp_title = we don,t  need employee job profile name to find if he leave job or not
* revol_bal = we aalready have revol_utiil we basically tell aabout revol_bal in percentage
* title = it is same as purpose
* grade = we had better explained column name sub_grade
* Emp_length = this

In [ ]:
cols_to_drop = ["emp_title","revol_bal","title","grade","emp_length","address"]
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df = df.drop(existing_cols_to_drop, axis=1)

# Treat Null values of each columns required  

In [ ]:
df['revol_util_missing'] = df['revol_util'].isna().astype(int)
df['revol_util'] = df['revol_util'].fillna(df['revol_util'].median())
df['mort_acc_missing'] = df['mort_acc'].isna().astype(int)
df['mort_acc'] = df['mort_acc'].fillna(0)
df['pub_rec_bankruptcies_missing'] = df['pub_rec_bankruptcies'].isna().astype(int)
df['pub_rec_bankruptcies'] = df['pub_rec_bankruptcies'].fillna(0)

In [ ]:
df.isna().sum()

# Exploratory Data Analysis  

In [ ]:
df["loan_status"].value_counts(normalize=True)*100

In [ ]:
df.groupby("purpose")["loan_status"].value_counts(normalize=True)*100

In [ ]:
filtered_df = df[df['purpose'].isin(["credit_card", "debt_consolidation", "home_improvement","major_purchase","other"])]
filtered_df.groupby("loan_status")["purpose"].value_counts(normalize=True)*100

In [ ]:
total_chharged_off=(df["loan_status"] == "Charged Off").sum()

In [ ]:
total_filtered=filtered_df["loan_status"].value_counts().loc["Charged Off"]
percent=total_filtered/total_chharged_off *100
percent

In [ ]:
df=df.drop("installment",axis=1)

In [ ]:
filtered_df_debt = df[df['purpose'].isin(["debt_consolidation"])]
filtered_df_debt=filtered_df_debt.groupby("loan_status")["purpose"].value_counts()
percent_of_debt_consolidation=(filtered_df_debt/total_chharged_off*100)
percent_of_debt_consolidation

In [ ]:
df.groupby("loan_status")["purpose"].value_counts(normalize=True)*100

93% of load default happens for the people who had purpose of credit card,debt_consolidation,home_improvement,other and major_purchase

* 62% of total chargge off is contributed by only debt_consolidation purpose.


In [ ]:
df.groupby("sub_grade")["loan_status"].value_counts(normalize=True)*100

# Treat outlier for annual income.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def compare_outliers(df, column):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.boxplot(y=df[column], ax=axes[0])
    axes[0].set_title(f"Before Treatment - {column}")

    # IQR method
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    df[column] = np.where(df[column] < lower_limit, lower_limit,
                          np.where(df[column] > upper_limit, upper_limit, df[column]))

    sns.boxplot(y=df[column], ax=axes[1])
    axes[1].set_title(f"After Treatment - {column}")
    plt.show()
compare_outliers(df, 'annual_inc')


In [ ]:
df.groupby(["application_type"])["loan_status"].value_counts(normalize=True)*100

* THEIR IS A RISK IN DIRECT PAY which have 35% rate of charge off

* direct pay  have 35 % of charge off. which is significantly high


In [ ]:
df.columns

In [ ]:
df.groupby("application_type").nunique()

In [ ]:
df.groupby("verification_status")["loan_status"].value_counts(normalize=True)*100

* person who had verification_status as Not Verified had less charged off rate

In [ ]:
df[["pub_rec", "pub_rec_bankruptcies"]] = df[["pub_rec", "pub_rec_bankruptcies"]].apply(pd.to_numeric, errors="coerce")
# Calculate correlation
correlation = df["pub_rec"].corr(df["pub_rec_bankruptcies"])
print(f"Correlation: {correlation:.4f}")

# Feature Engineering

In [ ]:
df["Loan-to-Income Ratio"]=df["loan_amnt"]/df["annual_inc"]

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')
df['credit_experience_years'] = (df['issue_d'].dt.year - df['earliest_cr_line'].dt.year) + \
                                (df['issue_d'].dt.month - df['earliest_cr_line'].dt.month)/12

# define experience categories
def experience_category(years):
    if years <= 1:
        return "Fresher"
    elif years <= 2:
        return "Beginner"
    elif years <= 5:
        return "Medium Experience"
    else:
        return "Very Experienced"

# apply function
df['credit_experience_level'] = df['credit_experience_years'].apply(experience_category)

# check result
print(df[['issue_d','earliest_cr_line','credit_experience_years','credit_experience_level']].head(10))
df=df.drop(['issue_d','earliest_cr_line','credit_experience_years'],axis=1)

In [ ]:
df.groupby("credit_experience_level")["loan_status"].value_counts(normalize=True)*100


In [ ]:
def experience_category(years):
    if  year<= []:
        return "Fresher"
    elif years <= 2:
        return "Beginner"
    elif years <= 5:
        return "Medium Experience"
    else:
        return "Very Experienced"

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
print("Numeric columns:")
print(numeric_cols)

In [ ]:
numeric_df = df.select_dtypes(include=np.number)
plt.figure(figsize=(10,8))
sns.heatmap(data=numeric_df.corr())

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df

In [ ]:
df.groupby("int_rate")["loan_status"].value_counts(normalize=True).sort_values(ascending=False)

In [ ]:
def categroy(int_rate):
    if int_rate >= 0.0 and int_rate < 5.0:
        return "Low"
    elif int_rate >= 5.0 and int_rate < 10.0:
        return "normmal rate "
    elif int_rate >= 10.0 and int_rate < 15.0:
        return "Standard"
    elif int_rate >= 15.0 and int_rate < 20.0:
        return "high"
    elif int_rate >= 20.0 and int_rate < 25.0:
        return "very high"
    else:
        return "Extrme"
df["int_rate_category"]= df["int_rate"].apply(categroy)

In [ ]:
df

In [ ]:
df.groupby("loan_status")["int_rate_category"].value_counts(normalize=True)*100

charge of rate is very high when interest rate is between 10% to 20% . almost 74% of total ccharge off belongs to this interest rate only  

In [ ]:
# Find actual max annual income
max_income = df["annual_inc"].max()

# Ensure the last bin edge is greater than all previous ones
bins = [0,25000,50000,75000,100000,150000,200000,500000,max_income]

# Drop duplicates just in case
bins = sorted(set(bins))

# Define labels = len(bins)-1
labels = ["Entry_level","Junior","Mid-level","Experienced",
          "Senior","Lead","Director","Executive"]

# Apply pd.cut
df["income_category"] = pd.cut(df["annual_inc"], bins=bins, labels=labels, include_lowest=True)

# Check
print(df[["annual_inc","income_category"]].head(10))


In [ ]:
df.rename(columns={"annual_inc":"Annual_inc"},inplace=True)